In [56]:
#Imports
import numpy as np
import pandas as pd
import json
import pickle

from gensim.models import Word2Vec
from gensim.models import Doc2Vec
import gensim
from gensim.models.doc2vec import TaggedDocument
import re

from IPython.core.display import display, HTML

import HTMLSentenceTokenizer
sentence = HTMLSentenceTokenizer.HTMLSentenceTokenizer()
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import ast

import tqdm
from tqdm import tqdm_notebook

import functions 
clean = functions.clean()

In [36]:
att = pd.read_csv("/Users/conorosully/Documents/Legal-Case-Prediction/data/preclean/case_attributes.csv",'#')
print(att.columns)
print(len(att))
doc_ids = att['id']

path = "/Users/conorosully/Documents/Legal-Case-Prediction/data/clean/{}.json"

with open(path.format('train_test_split'), 'r') as readfile:
    train_test_split = json.load(readfile)
    readfile.close()


Index(['id', 'type', 'url', 'doc-name', 'doc-id-label', 'doc-date', 'doc-type',
       'type-description', 'vl-type', 'created-at', 'updated-at',
       'app-no-parts', 'app-nos', 'articles', 'conclusions', 'courts', 'ecli',
       'judgement-date', 'judges', 'last-modified-time', 'nonviolations',
       'organisations', 'respondents', 'separate-opinion', 'violations'],
      dtype='object')
56688


In [9]:
train_test_split['2'].keys()

dict_keys(['v', 'nv', 'v_train', 'nv_train', 'v_test', 'nv_test'])

In [30]:
#Documents to be split
# - Documents with ID in train or testing set

law_split = []
for article in ["2","3","5","6","8","10","13","14"]:
    IDs = train_test_split[article]
    law_split += IDs['v_train'] + IDs['nv_train'] + IDs['v_test']+ IDs['nv_test']
law_split = list(set(law_split))
len(law_split)

3840

In [44]:
#Documents to exclude
# - Summary with same name

names = att[att['id'].isin(law_split)]['doc-name']
summary = att[att['doc-type'] =='CLIN']
exclude = summary[summary['doc-name'].isin(names)]['id']
exclude = list(exclude)
len(exclude)

813

In [34]:
def splitLaw(case):
    """
    Returns case without law section
    """
    split_text = case.split('THE LAW',maxsplit=1)
    return split_text[0]
case = clean.getCase(law_split[0])[law_split[0]]
splitLaw(case)

'<div> <p align="center">FIFTH SECTION</p> <p align="center"></p> <p align="center"></p> <p align="center"><b>CASE OF </b><b>M.K. v. FRANCE</b></p> <p align="center"></p> <p align="center"><i>(Application no. 19522/09)</i></p> <p align="center"></p> <p align="center"></p> <p align="center">JUDGMENT</p> <p align="center"></p> <p align="center"></p> <p align="center">STRASBOURG</p> <p align="center"></p> <p align="center">18 April 2013</p> <p></p> <p align="center"><b><u>FINAL</u></b></p> <p align="center"></p> <p align="center">18/07/2013</p> <p align="center"></p> <p align="center"></p> <p class="sub-section">This judgment is final but it may be subject to editorial revision.</p> </div> <br><div> <p class="section">In the case of M.K. v. France</p> <p>The European Court of Human Rights (Fifth Section) sitting as a Chamber composed of:</p> <p>Mark Villiger<i> President</i><br>Angelika Nußberger<br>Ann Power-Forde<br>André Potocki<br>Paul Lemmens<br>Helena Jäderblom<br>Aleš Pejchal<i> ju

To avoide data leakage:
- Exclude the law and verdict section of any judgement used in train or test set. (3840)
- Exclude the sumamry of any judgemtn used in train or test set. 

All the other documents are used to create word2vec and doc2vec embeddingsm

# word2vec

In [ ]:
# - take case
# - if ID remove law
# - 3 stopword removal
# - dump each to a pickel file

In [57]:
path = "/Users/conorosully/Documents/Legal-Case-Prediction/data/preclean/{}"

with open(path.format('sentence_none.pickle'), 'wb') as f:
    for ID in tqdm_notebook(doc_ids):
        doc = clean.getCase(ID)[ID]

        if ID in law_split:
            doc = splitLaw(doc)

        if ID not in exclude:
            parsed_sentences = sentence.feed(doc)
            for sent in parsed_sentences:
                tokens = word_tokenize(sent)

                tokens = [word for word in tokens if word.isalpha()]
                tokens = [w.lower() for w in tokens]
                if len(tokens) > 2:
                    pickle.dump(tokens,f)
                
    f.close()

In [58]:
#To load from pickle file
sentences = []
with open(path.format('sentence_none.pickle'), 'rb') as fr:
    try:
        while True:
            sentences.append(pickle.load(fr))
    except EOFError:
        pass

print(len(sentences))
#print(documents[0])

4795918


In [ ]:
"""stopwords = stopwords.words('english')

path = "/Users/conorosully/Documents/Legal-Case-Prediction/data/preclean/{}"

with open(path.format('sentence_english.pickle'), 'wb') as f:
    tokens = [word for word in tokens if word not in stopwords]"""

In [60]:
count = 0
for s in tqdm_notebook(sentences):
    count += len(s)
print(count) #Number of unique tokens


80757041


#### Create embeddings

In [67]:
path = "/Users/conorosully/Documents/Legal-Case-Prediction/data/embeddings/{}"

for s in [100,200]:
    # train model
    model = Word2Vec(sentences, min_count=10,size=s)

    words = list(model.wv.vocab)
    #print(model['case'])
    name = 'echt2vec_{}.txt'.format(s)
    model.save(path.format(name))
    print(s)

/Users/conorosully/miniconda3/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


100
200


In [ ]:
new_model = Word2Vec.load(path.format('echt2vec_100.txt'))
print(new_model)
print(new_model['the'])

# doc2vec

In [ ]:
#GOOGLE BEST PRACTISES

In [ ]:
# Create documents from all judgements with 3 versions: none, english and legal
# - For each text json
# - If ID: exclude else full
# - dump to file

In [64]:
#create docuemnts
with open(path.format('documents_none.pickle'), 'wb') as f:
    
    for ID in tqdm_notebook(doc_ids):
        
        doc = clean.getCase(ID)[ID]
        
        if ID in law_split:
            doc = splitLaw(doc)

        if ID not in exclude:
            parsed_sentences = sentence.feed(doc)

            document = []
            for sent in parsed_sentences:

                tokens = word_tokenize(sent)
                tokens = [word for word in tokens if word.isalpha()]
                tokens = [w.lower() for w in tokens]
                document += tokens
        
            pickle.dump(document,f)
                
f.close()

In [65]:
#To load from pickle file
documents = []
count = 0
with open(path.format('documents_none.pickle'), 'rb') as fr:
    try:
        while True:
            documents.append(pickle.load(fr))
            if count%5000 == 0:
                print(count, end=" ")
            count += 1
    except EOFError:
        pass

print(len(documents))
#print(documents[0])

0 5000 10000 15000 20000 25000 30000 35000 40000 45000 50000 55000 55875


In [66]:
documents = [TaggedDocument(doc, [i]) 
             for i, doc in tqdm_notebook(enumerate(documents))]

We use these hyper-parameter values for WIKI (APNEWS): vector size = 300 (300), window size = 15 (15),
min count = 20 (10), sub-sampling threshold = 10−5
(10−5),
negative sample = 5, epoch = 20 (30). After removing low
frequency words, the vocabulary size is approximately 670K
for WIKI and 300K for AP-NEWS.
10That is, test data is held out and

In [68]:
# define training data
path = "/Users/conorosully/Documents/Legal-Case-Prediction/data/embeddings/{}"

for s in [100,200]:
    # train model
    model = Doc2Vec(documents, 
                    size=s, 
                    window=15, 
                    min_count=10, 
                    workers = 4,
                    epoch = 20,
                    negative = 5)
    name = 'doc2vec2_{}'.format(s)
    model.save(path.format(name))
    print(s)

/Users/conorosully/miniconda3/lib/python3.6/site-packages/gensim/models/doc2vec.py:366: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


100
200
